A última execução deste notebook foi em  18 de fevereiro de 2022, às 15∶02∶19.

In [1]:
import requests
from requests.adapters import HTTPAdapter, Retry
from bs4 import BeautifulSoup
from unidecode import unidecode
from string import digits
import re
import pandas as pd
import numpy as np

Aqui é realizado web scraping no site de quatro super mercados de Fortaleza para montagem de uma lista de nomes de produtos e suas categorias. O resultado final é armazenado em um arquivo csv.

# Função para tratamento dos nomes dos produtos
Primeiramente vamos criar uma função que transforma uma string deixando-a toda com caracteres minúsculos e
removendo dígitos, acentos e caracteres de pontuação. Também removeremos expressões que referem-se em unidades e
medidas, e não à natureza do produto em si.

In [80]:
pal_unidades = ['unidade','kg','caixa','litro','pacote','embalagem','frasco','cx','garrafa','preco','desconto',
                'vidro','bandeja','higienizado','higienizada','higienziada','higienizados','selecionada',
                'selecionado']
sg_unidades = ['g','un','l','ml','s']

def trata_str(string):
    regex = re.compile('[^a-zA-Z ]')
    nome_prod = regex.sub('',unidecode(string.translate(str.maketrans('', '', digits)).lower()))
    
    for sg in sg_unidades:
        nome_prod = nome_prod.replace(' ' + sg + ' ','')
        if nome_prod[(len(sg) +1) * -1:] == ' ' + sg:
            nome_prod = nome_prod[0:len(sg) * -1:]
    
    for palavra in pal_unidades:
        nome_prod = nome_prod.replace(palavra,'')
    
    nome_prod = re.sub(' +',' ',nome_prod) # Retirando espaços múltiplos.
    return nome_prod.strip()

# Dados de quatro supermercados situados em Fortaleza
Vamos ler os produtos disponívels em quatro supermercados para capturar categorias de produtos. Foram escolhidos mercados de Fortaleza pois a massa que estamos tratando trata-se de pedidos de feiras feitos nesta cidade, e podemos ter produtos ou termos específicos da região na nossa massa.

In [76]:
# Recebe uma lista de URLs e faz varredura de todos produtos nelas, retornando um dicionário com os produtos e suas
# categorias.
def busca_produtos1(lista_urls):
    headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0"}
    prods = dict()
    qt_erros = int()

    s = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
    s.mount('http://', HTTPAdapter(max_retries=retries))
    
    # Buscando categorias na páginas principal.
    for url_ent in lista_urls:
        url = url_ent + '/setores'
        print('-'*115)
        print(f"Tratando {url_ent}")
        print('-'*115)
        response = s.get(url,headers=headers)

        if response.ok:
            html = response.content.decode()
            soup = BeautifulSoup(html,'html.parser')
            links = soup.find_all('li')
    
            nomes_categorias = list()
            links_categorias = list()

            for link in links:
                nomes_categorias.append(link.get('id'))
                link_parcial = link.find_all('a')[0].get('href')
                links_categorias.append(url_ent + link_parcial[1:])
            print(f"Categorias lidas: {len(nomes_categorias)}.")
        else:
            print(f"ERRO {response.status_code} em {url}")
            qt_erros += 1
            
        # Buscando subcategorias nas páginas de categorias
        nomes_sub_cats = list()
        links_sub_cats = list()
        i = int()

        for link_cat in links_categorias:
            url = link_cat

            response = s.get(url,headers=headers)

            if response.ok:
                html = response.content.decode()
                soup = BeautifulSoup(html,'html.parser')
                links = soup.find_all('li')
    
                for link in links:
                    nomes_sub_cats.append(link.get('id'))
                    link_parcial = link.find_all('a')[0].get('href')
                    links_sub_cats.append(url_ent + link_parcial[1:])
                print(f"Tratou {nomes_categorias[i]}. Subcategorias lidas: {len(links)}.")
                i += 1
            else:
                print(f"ERRO {response.status_code} em {link_cat}")
                qt_erros += 1

        # Buscando produtos nas páginas de subcategorias
        sub_cats = list(zip(nomes_sub_cats,links_sub_cats)) # pos[0]: nome; pos[1]: link
        
        for sub_cat in sub_cats:
            url = sub_cat[1]
            response = s.get(url,headers=headers)

            if response.ok:
                html = response.content.decode()
                soup = BeautifulSoup(html,'html.parser')
                produtos = soup.find_all('div', class_='sc-eCssSg dCfzHX')
                for produto in produtos:
                    prods[trata_str(produto.text)] = sub_cat[0].split('/')[-1].replace('-',' ')
                print(f"Tratou {sub_cat[0]}. Produtos lidos: {len(produtos)}.")
            else:
                print(f"ERRO {response.status_code} em {sub_cat[1]}")
                qt_erros += 1
                
    print('-'*115)
    print('Final de processamento')
    print(f"Erros ao acessar urls: {qt_erros}")
    print(f"Total de produtos lidos: {len(prods)}")
    print('-'*115)
    return prods

dic_prods1 = busca_produtos1(['https://amercado.americanas.com.br/produtos/centerbox-conceito',
                           'https://amercado.americanas.com.br/produtos/hipermercado-big-bompreco-bezerra-de-menezes',
                           'https://amercado.americanas.com.br/produtos/pao-de-acucar-nautico',
                           'https://amercado.americanas.com.br/produtos/mercado-extra-rodoviaria'])


-------------------------------------------------------------------------------------------------------------------
Tratando https://amercado.americanas.com.br/produtos/centerbox-conceito
-------------------------------------------------------------------------------------------------------------------
ERRO 503 em https://amercado.americanas.com.br/produtos/centerbox-conceito/setores


UnboundLocalError: local variable 'links_categorias' referenced before assignment

# Captura de dados em sites específicos de feiras
Como os produtos ofertados em supermercados não são exatamente iguais aos ofertados em feiras, vamos acrescentar informação coletadas em sites específicos de feiras à nossa massa.

## Sos Feira
Fonte: https://www.sosfeira.com.br/.

In [81]:
def busca_produtos2(lista_urls):
    headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0"}
    prods = dict()
    qt_erros = int()

    s = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
    s.mount('http://', HTTPAdapter(max_retries=retries))
    
    for url_ent in lista_urls:
        print(f"Tratando {url_ent}")
        response = s.get(url_ent,headers=headers)

        if response.ok:
            html = response.content.decode()
            soup = BeautifulSoup(html,'html.parser')
            
            categ = soup.find_all('h1', class_='titulo cor-secundaria')
            categoria = categ[0].text
            
            produtos = soup.find_all('div', class_='imagem-produto has-zoom')
            for produto in produtos:
                produto = produto.find_all('img')[0].get('alt')
                
                produto = trata_str(produto)
                categoria = trata_str(categoria)
                prods[produto] = categoria
        else:
            print(f"ERRO {response.status_code} em {url_ent}")
            qt_erros += 1

    print('-'*115)
    print('Final de processamento')
    print(f"Erros ao acessar urls: {qt_erros}")
    print(f"Total de produtos lidos: {len(prods)}")
    print('-'*115)
    return prods


dic_prods2 = busca_produtos2(['https://www.sosfeira.com.br/frutas',
                            'https://www.sosfeira.com.br/verduras',
                            'https://www.sosfeira.com.br/legumes'])

Tratando https://www.sosfeira.com.br/frutas
Tratando https://www.sosfeira.com.br/verduras
Tratando https://www.sosfeira.com.br/legumes
-------------------------------------------------------------------------------------------------------------------
Final de processamento
Erros ao acessar urls: 0
Total de produtos lidos: 45
-------------------------------------------------------------------------------------------------------------------


## Verduranet
Fonte: https://www.verduranet.com/.

In [83]:
def busca_produtos3(lista_urls):
    headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:97.0) Gecko/20100101 Firefox/97.0"}
    prods = dict()
    qt_erros = int()

    s = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
    s.mount('http://', HTTPAdapter(max_retries=retries))
    
    for url_ent in lista_urls:
        url = url_ent
        pag = 1
        response = s.get(url,headers=headers)

        while response.ok:
            print(f"Tratando {url}")
            html = response.content.decode()
            soup = BeautifulSoup(html,'html.parser')
            
            categ = soup.find_all('nav', class_='woocommerce-breadcrumb')[0]
            categoria = categ.text.replace('\t','').replace('\n','').replace('\r','').split('/')[1]
            
            produtos = soup.find_all('div', class_='product-name')
            for produto in produtos:
                produto = produto.find_all('h2')[0].text
                
                produto = trata_str(produto)
                categoria = trata_str(categoria)
                prods[produto] = categoria
                
            pag += 1
            url = url_ent + 'page/' + str(pag)
            response = s.get(url,headers=headers)
        else:
            print(f"Final de categoria com {url}")

    print('-'*115)
    print('Final de processamento')
    print(f"Total de produtos lidos: {len(prods)}")
    print('-'*115)
    return prods


dic_prods3 = busca_produtos3(['https://www.verduranet.com/product-category/vegetais/',
                              'https://www.verduranet.com/product-category/frutas/',
                              'https://www.verduranet.com/product-category/saladas/',
                              'https://www.verduranet.com/product-category/temperos/'])

Tratando https://www.verduranet.com/product-category/vegetais/
Tratando https://www.verduranet.com/product-category/vegetais/page/2
Tratando https://www.verduranet.com/product-category/vegetais/page/3
Tratando https://www.verduranet.com/product-category/vegetais/page/4
Final de categoria com https://www.verduranet.com/product-category/vegetais/page/5
Tratando https://www.verduranet.com/product-category/frutas/
Tratando https://www.verduranet.com/product-category/frutas/page/2
Final de categoria com https://www.verduranet.com/product-category/frutas/page/3
Tratando https://www.verduranet.com/product-category/saladas/
Final de categoria com https://www.verduranet.com/product-category/saladas/page/2
Tratando https://www.verduranet.com/product-category/temperos/
Final de categoria com https://www.verduranet.com/product-category/temperos/page/2
-------------------------------------------------------------------------------------------------------------------
Final de processamento
Total de 

# Unificação dos produtos lidos.
Além de unificar todos os produtos em um único dicionário, vamos criar um dicionário com alguns livros. Livros estão presentes nos dados que iremos tratar, entretanto não são vendidos nas fontes que consultamos.

In [87]:
dic_prods9 = {'livro gato de botas':'livros',
             'livro moby dick': 'livros',
             'livro as longas trancas de um careca':'livros',
             'livro a volta dos que nao foram':'livros',
             'livro mamae nao quer que eu case':'livros',
             'livro as aventuras de um aventureiro':'livros',
             'livro as desventuras de um azarado':'livros',
             'livro o cachorro sapeca':'livros',
             'livro o gato que miava au au':'livros',
             'livro o suspiro do doceiro':'livros',
             'livro as desventuras de um azarado':'livros',
             'livro as fronteiras que unem':'livros',}

dic_prods = dic_prods1 | dic_prods2 | dic_prods3 | dic_prods9

NameError: name 'dic_prods1' is not defined

# Análise das categorias levantadas
Vamos ver se existem categorias muito parecidas ou inócuas. Caso positivo, trataremos para termos dados com a melhor qualidade possível.

In [ ]:
df= pd.DataFrame.from_dict(dic_prods,orient='index')
df.reset_index(inplace=True)
df.columns = ['produto','categoria']

In [ ]:
np.sort(df['categoria'].unique())

Faremos as seguintes intervenções:
- remover "espaco mondelez";
- mudar "ajinomoto" para "temperos e condimentos";
- mudar "categoria maquiagem" para "maquiagem";
- mudar "cervejas ambev" para "cervejas";
- mudar "danone" para "iogurtes e coalhadas";
- mudar "festival do bebe carrefour" para "higiene";
- mudar "higiene bebe" para "higiene";
- mudar "johnson and johnson" para "higiene";
- mudar "hortifruti manipulados" para "hortifruti";
- mudar "mais hortifruti" para "hortifruti";
- mudar "mars" para "petshop";
- mudar "petshop aves" para "petshop";
- mudar "petshop caes" para "petshop";
- mudar "petshop gatos" para "petshop";
- mudar "papel aluminio papel manteiga" para "embalagens";
- mudar "filme pvc sacos plasticos" para "embalagens";
- mudar "padaria fabricacao propria" para "paes";
- mudar "prudence" para "preservativos correlatos";
- mudar "pratos prontos" para "pronto para consumo";
- mudar "sobremesas lacteas" para "sorvetes sucos e sobremesas";
- mudar "whisky" para "destilados";
- mudar "drinks" para "destilados";
- mudar "doces padaria" para "doces e sobremesas";
- mudar "embutidos finos" para "frios e embutidos";
- mudar "outros utilidades domesticas" para "uso geral".

In [ ]:
len(df)

In [ ]:
df.drop(df[df['categoria'] == 'espaco mondelez'].index,inplace=True)

trocas = {"ajinomoto":"temperos e condimentos",
"categoria maquiagem":"maquiagem",
"cervejas ambev":"cervejas",
"danone":"iogurtes e coalhadas",
"festival do bebe carrefour":"higiene",
"higiene bebe":"higiene",
"johnson and johnson":"higiene",
"hortifruti manipulados":"hortifruti",
"mais hortifruti":"hortifruti",
"mars":"petshop",
"petshop aves":"petshop",
"petshop caes":"petshop",
"petshop gatos":"petshop",
"papel aluminio papel manteiga":"embalagens",
"filme pvc sacos plasticos":"embalagens",
"padaria fabricacao propria":"paes",
"prudence":"preservativos correlatos",
"pratos prontos":"pronto para consumo",
"sobremesas lacteas":"sorvetes sucos e sobremesas",
"whisky":"destilados",
"drinks":"destilados",
"doces padaria":"doces e sobremesas",
"embutidos finos":"frios e embutidos",
"outros utilidades domesticas":"uso geral"}

df['categoria'].replace(trocas,inplace=True)

# Salvando os dados em arquivo .csv

In [ ]:
df.sort_values(by='produto',inplace=True) # Para misturar os produtos dos diferentes mercados entre si.
df.to_csv('arquivos/produtos_categorias.csv',index=False)